In [1]:
import numpy as np
import csv

import pyspark.mllib.regression 
from pyspark import SparkConf, SparkContext
from pyspark.ml.classification import RandomForestClassifier 
from pyspark.sql import SparkSession
from pyspark.mllib.regression import LabeledPoint
from sklearn.model_selection import train_test_split
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from sklearn import preprocessing
from pyspark.ml.linalg import DenseVector
from pyspark.ml.regression import LinearRegression
import config as cf

In [2]:
NUM_FEATURES = 10
NUM_DATAPOINTS = 100000
CLASSES = {
    'Very Large Number': 1,
    'Large Number': 1,
    'Medium Number': 0,
    'Small Number': 0,
    'Very Small Number': 0}

In [3]:
def create_spark_context():
    """Set up spark context"""
    conf = (SparkConf())
    sc = SparkContext(conf=conf)

    return sc

In [4]:
sc = create_spark_context()

In [5]:
X_unscaled = []
y_class = []
y_reg_unscaled = []
#Read data in from csv file and store as matrix and column vectors
with open(cf.PATHS[cf.SUM_WITH_NOISE]) as csvfile:
    reader = csv.reader(csvfile)
    for i, row in enumerate(reader):         
        if(i != 0):
            row = row[0].split(';')
            xi = [None]*NUM_FEATURES
            xi[0] = row[1]
            xi[1] = row[2]
            xi[2] = row[3]
            xi[3] = row[4]
            xi[4] = row[5]
            xi[5] = row[6]
            xi[6] = row[7]
            xi[7] = row[8]
            xi[8] = row[9]
            xi[9] = row[10]
            y_reg_unscaled.append(row[11])
            y_class.append(CLASSES[row[12]])
            X_unscaled.append(xi)
            
        if(i == NUM_DATAPOINTS):
            break
#Standardise the data
X = preprocessing.scale(X_unscaled)

In [8]:
class LinReg:
    """A class that takes in a numpy matrix X and column vector y 
        Allows you to apply 7030 split and 10 fold cross fold validation
        with metrics returned as a tuple as so:
        (RMSE, MAE)
    """
    def __init__(self, X, y):
        self.spark = SparkSession(sc)
        self.X = X
        #Standardise y values
        self.y = preprocessing.scale(y)
        self.data = []
        #Convert to string for ease of construct for the RDD
        for i, yi in enumerate(self.y):
            xi = self.X[i]
            xi_string = ""
            xi_string+=str(yi)
            for x in xi:
                xi_string+=','
                xi_string+=str(x)
            self.data.append(xi_string)

        self.rdd = sc.parallelize(self.data)
        self.rdd = self.rdd.map(lambda line: line.split(","))
        #Convert to df for easy manipulation
        self.df = self.rdd.map(lambda line: Row(target=line[0],
                              f1=line[1], 
                              f2=line[2], 
                              f3=line[3],
                              f4=line[4], 
                              f5=line[5], 
                              f6=line[6],
                              f7=line[7], 
                              f8=line[8], 
                              f9=line[9],
                              f10=line[10])).toDF()
        self.columns = ['target','f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10']
        self.df = self.convertColumn(self.df, self.columns, FloatType())
        self.input_data = self.df.rdd.map(lambda x: (x[10], DenseVector(x[:10] )))
        self.df1 = self.spark.createDataFrame(self.input_data, ["label", "features"])
        
    def convertColumn(self, df, names, newType):
        """Convert string columns to float columns"""
        for name in names: 
            df = df.withColumn(name, df[name].cast(newType))
        return df 
    
    def split7030(self):
        """Apply 70 30 split testing"""
        train_data, test_data = self.df1.randomSplit([.7,.3],seed=1234)
        # Initialize `lr`
        lr = LinearRegression(labelCol="label", maxIter=10)

        # Fit the data to the model
        linear_model = lr.fit(train_data)
        linear_model.transform(test_data)
        metric_1 = linear_model.summary.rootMeanSquaredError
        metric_2 = linear_model.summary.meanAbsoluteError
        return metric_1, metric_2
    
    def crossval10(self):
        """Apply 10 fold cross validation testing"""
        metric_1_sum = 0
        metric_2_sum = 0
        for i in range(0, 10):
            train_data, test_data = self.df1.randomSplit([.9,.1],seed=i*1234)
            self.lr = LinearRegression(labelCol="label", maxIter=10)
            self.linear_model = self.lr.fit(train_data)
            self.linear_model.transform(test_data)
            metric_1 = self.linear_model.summary.rootMeanSquaredError
            metric_1_sum += metric_1
            metric_2 = self.linear_model.summary.meanAbsoluteError
            metric_2_sum += metric_2
            
            
        return metric_1_sum/10, metric_2_sum/10

In [9]:
lr = LinReg(X, y_reg_unscaled)

In [10]:
result = lr.split7030()
print("Root mean square error: ", result[0])
print("Mean absolute error: ", result[1])

Root mean square error:  0.11975642411407696
Mean absolute error:  0.08961014980816806


In [16]:
result = lr.crossval10()
print("Root mean square error: ", result[0])
print("Mean absolute error: ", result[1])

Root mean square error:  0.11995015298624785
Mean absolute error:  0.08981624145365084


In [12]:
class RandF:
    """A class that takes in a numpy matrix X and column vector y 
        and implements the random forest algorithm with a max depth of 2
        Allows you to apply 7030 split and 10 fold cross fold validation
        with metrics returned as a tuple as so:
        (f1 score, accuracy)"""
    def __init__(self, X, y):
        self.spark = SparkSession(sc)
        self.X = X
        self.y = y
        self.data = []
        for i, yi in enumerate(self.y):
            xi = self.X[i]
            xi_string = ""
            xi_string+=str(yi)
            for x in xi:
                xi_string+=','
                xi_string+=str(x)
            self.data.append(xi_string)

        self.rdd = sc.parallelize(self.data)
        self.rdd = self.rdd.map(lambda line: line.split(","))
        self.df = self.rdd.map(lambda line: Row(target=line[0],
                              f1=line[1], 
                              f2=line[2], 
                              f3=line[3],
                              f4=line[4], 
                              f5=line[5], 
                              f6=line[6],
                              f7=line[7], 
                              f8=line[8], 
                              f9=line[9],
                              f10=line[10])).toDF()
        self.columns = ['target','f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10']
        self.df = self.convertColumn(self.df, self.columns, FloatType())
        self.input_data = self.df.rdd.map(lambda x: (x[10], DenseVector(x[:10] )))
        self.df1 = self.spark.createDataFrame(self.input_data, ["label", "features"])
        
    def convertColumn(self, df, names, newType):
        for name in names: 
            df = df.withColumn(name, df[name].cast(newType))
        return df 
    
    def split7030(self):
        """Apply 70 30 split testing"""
        train_data, test_data = self.df1.randomSplit([.7,.3],seed=1234)
        rf = RandomForestClassifier(labelCol='label', featuresCol='features',numTrees=3)
        fit = rf.fit(train_data)
        predicted = fit.transform(test_data)
        predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
        labels = predicted.select("label").rdd.map(lambda x: x[0])
        prediction_and_label = predictions.zip(labels).collect()
       
        return self.metrics(prediction_and_label)
        
    
    def crossval10(self):
        """Apply 10 fold cross validation"""
        metric_1_sum = 0
        metric_2_sum = 0
        for i in range(0, 10):
            train_data, test_data = self.df1.randomSplit([.9,.1],seed=i*1234)

            rf = RandomForestClassifier(labelCol='label', featuresCol='features',numTrees=3)
            fit = rf.fit(train_data)
            predicted = fit.transform(test_data)

            predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
            labels = predicted.select("label").rdd.map(lambda x: x[0])
            prediction_and_label = predictions.zip(labels).collect()
            m = self.metrics(prediction_and_label)
            metric_1_sum += m[0]
            metric_2_sum += m[1]
            
        return metric_1_sum/10, metric_2_sum/10     
            
    def metrics(self, prediction_and_label):
        """returns f1 score and accuracy as tuple from an array of (predication, label) tuples"""

        TP = 0
        FP = 0
        TN = 0
        FN = 0
        for p, l in prediction_and_label:
            p_bool = 0
            if(p>0.5):
                p_bool = 1
            if(p_bool == 1 and l ==1):
                TP+=1
            if(p_bool == 1 and l ==0):
                FP+=1
            if(p_bool == 0 and l ==0):
                TN+=1
            if(p_bool == 0 and l ==1):
                FN+=1
                
        accuracy = float(TP+TN)/float(TP+FP+TN+FN)        
        recall = float(TP)/float(TP+FN)
        precision = float(TP)/float(TP+FP)
        f1 = 2*(recall*precision)/(recall + precision)
        return f1, accuracy   

In [13]:
rf = RandF(X, y_class)

In [14]:
result = rf.split7030()
print('70 30 split')
print('f1 score: ', result[0])
print('accuracy: ', result[1])

70 30 split
f1 score:  0.9968501523845197
accuracy:  0.9938527994683503


In [15]:
result = rf.crossval10()
print('10 fold cross validation')
print('f1 score: ', result[0])
print('accuracy: ', result[1])

10 fold cross validation
f1 score:  0.9967683300965937
accuracy:  0.9936936602452546


In [ ]:
sc.stop()